# Process Zoom Transcripts for Qualitative Interviews

Process single vtt files (produced from zoom recordings) to paragraphs.
Stepes include:
- combine different lines said by the same person to one paragraph
- anonymize zoom names

Files in the same folder:
- A csv with the first column as zoom names and the second column as code names
- vtt files produced from zoom recordings

In [1]:
import webvtt 
#run '!pip install webvtt-py' if haven't installed
import datetime

In [2]:
# read individual vtt files
file_name = 'XXXX.vtt'

#read transcript
transcript = open(file_name, "r").read()

#breakdown to each line
transcript = transcript.split("\n\n")

In [3]:
lines = []

for line in transcript[1:]: #remove first 'WEBVTT'
    each_line = line.split('\n')
    
    try:
        words = each_line[2].split(': ')
        time = each_line[1].split(' --> ')
        lines.append([words[0],words[1],time])
    except:
        pass # doing nothing on exception

In [5]:
#combine lines if from the same 
transcript_comb = []
# init_time = datetime.datetime.strptime('00:00:00.0','%H:%M:%S.%f')
end_time = datetime.datetime.strptime('00:00:00.0','%H:%M:%S.%f')

for i in range(len(lines)):
        if i>0:
            temp_content = lines[i-1][1]
            init_time = datetime.datetime.strptime(lines[i-1][2][0],'%H:%M:%S.%f')
            if lines[i][0] == lines[i-1][0]:
                temp_content = temp_content + ' ' + lines[i][1]          
            else:
                end_time = datetime.datetime.strptime(lines[i-1][2][1],'%H:%M:%S.%f')
                elapsed_time = (end_time - init_time).total_seconds()
                transcript_comb.append([lines[i][0], lines[i-1][2][0], lines[i-1][2][1], elapsed_time, temp_content])

In [7]:
# Code to anonymize participants
# read anonymize.csv as a dictionary
# first column is the zoom names
# second column is the code names

import csv

with open('anonymize.csv', mode='r') as infile:
    reader = csv.reader(infile)
    lookup_dict = {rows[0]:rows[1] for rows in reader}

In [8]:
# substitute Zoom names with Code names

for i in range(len(transcript_comb)):
    if transcript_comb[i][0] in lookup_dict.keys():
        key = transcript_comb[i][0]
        transcript_comb[i][0] = lookup_dict[key]

In [11]:
fields = ['Person','Initial Timestamp', 'End Timestamp','Duration','Transcript']


with open('XXXX_new', 'w') as f:
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(fields)
    write.writerows(transcript_comb)